In [1]:
!pip install h2o

     |████████████████████████████████| 177.0 MB 14 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=8d0b9d2b54c9f78251ea2a6aa551398ea3d9a686cec8f8ff930241e34961c469
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [2]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2xi4wkrp
  JVM stdout: /tmp/tmp2xi4wkrp/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp2xi4wkrp/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_dibn1q
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [5]:
test, train = dataSetup(1566)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
train.shape

(909, 101)

In [7]:
x = train.columns
y = "class"
x.remove(y)

In [8]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [9]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_1_AutoML_1_20220502_230216


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 1,252 weights/biases, 29.4 KB, 10,301 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,100,Input,0.0,,,,,,,,,
1,,2,10,Rectifier,0.0,0.0,0.0,0.00442,0.001694,0.0,0.006212,0.130647,0.487537,0.066381
2,,3,10,Rectifier,0.0,0.0,0.0,0.003031,0.002442,0.0,0.002318,0.298592,0.985798,0.051211
3,,4,10,Rectifier,0.0,0.0,0.0,0.031421,0.175963,0.0,-0.023097,0.327343,0.989846,0.015704
4,,5,2,Softmax,,0.0,0.0,0.004878,0.006105,0.0,0.061795,1.730824,-0.000242,0.019152




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.26572670367309187
RMSE: 0.515486860815183
LogLoss: 0.726484954295098
Mean Per-Class Error: 0.49250544662309365
AUC: 0.49500121036068745
AUCPR: 0.5052440260071062
Gini: -0.009997579278625102

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3095302765937274: 


,,0,1,Error,Rate
0,0,13.0,446.0,0.9717,(446.0/459.0)
1,1,6.0,444.0,0.0133,(6.0/450.0)
2,Total,19.0,890.0,0.4972,(452.0/909.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-05-02 23:03:24,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-05-02 23:03:24,2.734 sec,27257 obs/sec,1.049505,1,954.0,0.521130,0.740180,-0.086411,0.495149,0.496111,1.414,0.50275
2,,2022-05-02 23:03:24,3.054 sec,31027 obs/sec,11.332233,11,10301.0,0.515487,0.726485,-0.063011,0.495001,0.505244,1.414,0.49725



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,V59,1.000000,1.000000,0.014930
1,V80,0.932559,0.932559,0.013923
2,V6,0.916505,0.916505,0.013683
3,V65,0.911731,0.911731,0.013612
4,V14,0.885139,0.885139,0.013215
5,V16,0.868828,0.868828,0.012971
6,V58,0.846197,0.846197,0.012634
7,V64,0.841565,0.841565,0.012564
8,V48,0.811097,0.811097,0.012110
9,V35,0.808281,0.808281,0.012067



See the whole table with table.as_data_frame()


In [10]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_1_AutoML_1_20220502_230216,0.545304,0.695888,0.520824,0.498911,0.501125,0.251127
StackedEnsemble_BestOfFamily_4_AutoML_1_20220502_230216,0.531832,0.694321,0.502448,0.488105,0.500474,0.250474
StackedEnsemble_BestOfFamily_3_AutoML_1_20220502_230216,0.528351,0.693936,0.506414,0.497821,0.500356,0.250356
StackedEnsemble_AllModels_2_AutoML_1_20220502_230216,0.525703,0.694536,0.505831,0.496732,0.500616,0.250616
DeepLearning_grid_3_AutoML_1_20220502_230216_model_1,0.52116,0.692616,0.516703,0.5,0.49973,0.24973
DeepLearning_grid_3_AutoML_1_20220502_230216_model_6,0.519705,0.692797,0.510532,0.497821,0.499821,0.249821
StackedEnsemble_AllModels_5_AutoML_1_20220502_230216,0.513517,0.761593,0.509218,0.496754,0.527527,0.278284
DeepLearning_grid_1_AutoML_1_20220502_230216_model_9,0.512263,0.696235,0.490404,0.5,0.501533,0.251536
StackedEnsemble_AllModels_4_AutoML_1_20220502_230216,0.508223,0.695951,0.493741,0.491394,0.501307,0.251309
XRT_1_AutoML_1_20220502_230216,0.506212,0.909044,0.4887,0.485926,0.557126,0.310389
